In [17]:
import pandas as pd # standard
import numpy as np # standard
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy




import warnings
warnings.filterwarnings("ignore")

In [18]:
df = pd.read_excel("/Users/avery/OneDrive/Documents/GitHub/Clinical_TLB_2023-2024/lung_cancer_tlb.xlsx")
df['CancerType'] = np.where(df['CancerType'].isna(), 'Control', df['CancerType'])

In [19]:

# replace NA with control

# get location of cut off values
lower_column_index = df.columns.get_loc("T51")
upper_column_index = df.columns.get_loc("T83.1")
label_column_index = df.columns.get_loc("CancerType")

column_indices = np.arange(lower_column_index, upper_column_index)
column_indices = np.append(column_indices, 0)
column_indices = np.append(column_indices, 1)



column_indices = np.append(column_indices, label_column_index)

df = df.iloc[:, column_indices]

# keep only Control and Adenocarcinoma for analysis
df_tree = df[(df['CancerType'] == 'Control') | (df['CancerType'] == 'Squamous')]
df_tree["CancerType"] = np.where(df_tree["CancerType"] == "Squamous", 1, 0)
labels = df_tree["CancerType"]
df_tree = df_tree.reset_index(drop=True)
df_tree = df_tree.drop(["sample_id", "pub_id", "CancerType"], axis = 1)



array([[0.02128, 0.02093, 0.02077, ..., 0.01629, 0.0144, 'Squamous'],
       [0.02964, 0.02936, 0.02847, ..., 0.02167, 0.02125, 'Squamous'],
       [0.01942, 0.01944, 0.01942, ..., 0.01254, 0.01154, 'Squamous'],
       ...,
       [0.01588, 0.01602, 0.01609, ..., 0.02038, 0.01967, 'Control'],
       [0.01543, 0.01556, 0.01571, ..., 0.02111, 0.02022, 'Control'],
       [0.01511, 0.01535, 0.01547, ..., 0.01919, 0.01807, 'Control']],
      dtype=object)

In [20]:
features = df_tree.values
labels = labels.ravel()

In [21]:

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, n_estimators=500, max_depth=48, max_features="sqrt")

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(features, labels)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(features)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations